In [31]:
import cartopy.crs as ccrs
import rasterio
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from rasterio.plot import show

import folium
import geopandas as gpd

In [45]:
raster = rasterio.open('testTiff.tif')
lst = raster.read()
lst = lst.astype(float)
print(lst)

meta = raster.meta
meta.update(driver='GTiff')
meta.update(dtype=rasterio.float64)

with rasterio.open('lst.tif', 'w', **meta) as dst:
    dst.write(lst)

[[[    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  ...
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]]

 [[    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  ...
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]]

 [[    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  ...
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]
  [    0.     0.     0. ...     0.     0.     0.]]

 ...

 [[  255.   255.   255. ...   255.   255.   255.]
  [  255.   255.   2

In [46]:
m = folium.Map(location=[35.9452,14.3797], zoom_start=10)

boundsdata = r'C:/Users/Massimo/Documents/SchoolFolder/ThirdYear/Thesis/MassThesis/src/References/malta.geojson'
region = gpd.read_file(boundsdata)
nReserve_proj = region.to_crs({'init': 'epsg:4326'})

C:\Users\Massimo\anaconda3\envs\snap\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [47]:
with rasterio.open("lst.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, nReserve_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open('cropped.tif', 'w', **meta) as dst:
    dst.write(out_image)